<a href="https://colab.research.google.com/github/askot19/Analise_de_dados/blob/main/Telefonia/3%20-%20Modelos%20Preditivos/3%20-%20Modelos%20Preditivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align = 'left'>
<img src = 'https://i.imgur.com/8LTNXxF.jpg'>
</p>

---
## **Data Scientist & Analyst**

*by [Paulo Henrique de Melo Ferreira](https://www.linkedin.com/in/paulo-henrique-ferreira-20151374/)*

---

<p style="text-align: justify;">
O atual projeto visa analisar dados da <b>Alura Voz</b>, a qual consiste em uma empresa de telecomunicações. O objetivo desta análise é obter insights acerca do <b>Churn Rate</b> da empresa e encontrar fatores que estejam influenciando essa métrica.
Os dados desta análise foram obtidos através de uma <b>API</b> por webscrapping.

Na primeira etapa os dados foram trabalhandos, montados e limpos para que aqui fosse possível se realizar a análise e entendimento, principalmente do <b> Churn Rate</b>.

Na segunda etapa foram construídos gráficos no intuito de se entender o comportamento dos dados, se há alguma influência do gênero dos clientes, presença de dependentes ou algum outro fator associado. Ao final, foi realizada uma visualização das correlações entre as variáveis.

Nesta terceira e última etapa, faremos a construção de <b>Modelos Preditivos</b>, a fim de prever a evasão futura dos atuais clientes que compõem a estrutura da empresa. Para isso, realizaremos o método <b>One Hot Encoding (OHE)</b>, pois os algoritmos de Machine Learning não são capazes de trabalhar com textos, sendo assim precisamos converter as variáveis categóricas para uma grandeza capaz de ser analizada pelo modelo.

Estuda-se ainda a possibilidade de aplicarmos algum método de conferência de peso das variáveis como o Weight of Evidence (WOE).

Portando as principais características, podemos preparar os dados para treinamento e predição por meio de modelos estatísticos, dentre os quais serão utilizados a <b> Regressão Logística </b> e o modelo de <b> Random Forest </b>.

In [64]:
# Importação de pacotes para análise de dados
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [65]:
# Importando dados após a limpeza
dados = pd.read_csv('https://raw.githubusercontent.com/askot19/Projetos/main/Telefonia/Dados/aluravoz_clean.csv')
dados.head()

,ID,Evasao,Genero,Senior,Casado(a),Dependentes,Posse,ServicoTelefonico,MultiplasLinhas,ServicodeInternet,...,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StreamingTV,StreamingFilmes,Contrato,CobrancasemPapel,MetododePagamento,CobrancasMensais,CobrancasTotais
0,0002-ORFBO,Nao,Mulher,0,Sim,Sim,9,Sim,Nao,DSL,...,Sim,Nao,Sim,Sim,Nao,Anual,Sim,Correspondencia,65.6,593.30
1,0003-MKNFE,Nao,Homem,0,Nao,Nao,9,Sim,Sim,DSL,...,Nao,Nao,Nao,Nao,Sim,Mensal,Nao,Correspondencia,59.9,542.40
2,0004-TLHLJ,Sim,Homem,0,Nao,Nao,4,Sim,Nao,Fibra optica,...,Nao,Sim,Nao,Nao,Nao,Mensal,Sim,Eletronico,73.9,280.85
3,0011-IGKFF,Sim,Homem,1,Sim,Nao,13,Sim,Nao,Fibra optica,...,Sim,Sim,Nao,Sim,Sim,Mensal,Sim,Eletronico,98.0,1237.85
4,0013-EXCHZ,Sim,Mulher,1,Sim,Nao,3,Sim,Nao,Fibra optica,...,Nao,Nao,Sim,Sim,Nao,Mensal,Sim,Correspondencia,83.9,267.40


Para trabalharmos com modelos estatísticos, não podemos simplesmente utilizar variáveis categóricas, pois os algoritmos somente entendem valores numéricos. Dessa maneira precisamos realizar um tratamento das variáveis categóricas de interesse.

Como vimos anteriormente, a característica gênero não influencia na tomada de decisão entre permanecer ou não com os serviços apresentados pela Alura Voz, entretanto, mesmo que houvesse alguma diferença entre as categorias, não utilizaríamos essa característica a fim de evitar viéses equivocados acerca desses dados.

As demais categorias não apresentam possiblidade de enviesamento, então podemos analisar todas essas e avaliar o seu peso de contribuição para a variável de resposta.

Para a conversão das variáveis categóricas vamos utilizar o médoto **One Hot Encoding**. Que atribui cada categoria como uma coluna de um **Data Frame**, cada linha que este valor aparece no Data Frame original, ele recebe o valor 1 e, caso não apareça, recebe o valor 0. Dessa maneira pode-se considerar como o efeito de uma variável afeta a variável de resposta, sem atribuir uma ponderação à estatística da distribuição desses dados.

In [66]:
#Conferindo colunas
dados.columns

Index(['ID', 'Evasao', 'Genero', 'Senior', 'Casado(a)', 'Dependentes', 'Posse',
       'ServicoTelefonico', 'MultiplasLinhas', 'ServicodeInternet',
       'SegurancaOnline', 'BackupOnline', 'ProtecaodeDispositivo',
       'SuporteTecnico', 'StreamingTV', 'StreamingFilmes', 'Contrato',
       'CobrancasemPapel', 'MetododePagamento', 'CobrancasMensais',
       'CobrancasTotais'],
      dtype='object')

In [67]:
# Coletando colunas importantes para análise
features = ['ServicodeInternet', 'Contrato',
        'MetododePagamento']

In [68]:
# Construindo um novo data frame para executar as predições
df_ml = dados[features].copy()
df_ml.head()

,ServicodeInternet,Contrato,MetododePagamento
0,DSL,Anual,Correspondencia
1,DSL,Mensal,Correspondencia
2,Fibra optica,Mensal,Eletronico
3,Fibra optica,Mensal,Eletronico
4,Fibra optica,Mensal,Correspondencia


In [69]:
# Construindo One Hot Encoding
df_ohe = pd.get_dummies(df_ml)
df_ohe.head()

,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,Contrato_Anual,Contrato_Dois anos,Contrato_Mensal,MetododePagamento_Cartao de credito,MetododePagamento_Correspondencia,MetododePagamento_Debito automatico,MetododePagamento_Eletronico
0,1,0,0,1,0,0,0,1,0,0
1,1,0,0,0,0,1,0,1,0,0
2,0,1,0,0,0,1,0,0,0,1
3,0,1,0,0,0,1,0,0,0,1
4,0,1,0,0,0,1,0,1,0,0


In [70]:
# Criando coluna de testes
df_ohe['ID'] = 'none'

In [71]:
# Verificando Data Frame
df_ohe.head()

,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,Contrato_Anual,Contrato_Dois anos,Contrato_Mensal,MetododePagamento_Cartao de credito,MetododePagamento_Correspondencia,MetododePagamento_Debito automatico,MetododePagamento_Eletronico,ID
0,1,0,0,1,0,0,0,1,0,0,none
1,1,0,0,0,0,1,0,1,0,0,none
2,0,1,0,0,0,1,0,0,0,1,none
3,0,1,0,0,0,1,0,0,0,1,none
4,0,1,0,0,0,1,0,1,0,0,none


In [72]:
# Inserindo identificações
df_ohe['ID'] = dados['ID'].copy()

In [73]:
df_ohe['ID'].head()

0    0002-ORFBO
1    0003-MKNFE
2    0004-TLHLJ
3    0011-IGKFF
4    0013-EXCHZ
Name: ID, dtype: object

In [74]:
# Reinserindo dados acerca de pagamentos, posse e demais variáveis categóricas
df_ohe['CobrancasMensais'] = dados['CobrancasMensais'].copy()
df_ohe['CobrancasTotais'] = dados['CobrancasTotais'].copy()
df_ohe['Posse'] = dados['Posse'].copy()
df_ohe['Evasao'] = dados['Evasao'].copy()
df_ohe['Genero'] = dados['Genero'].copy()
df_ohe['Senior'] = dados['Senior'].copy()
df_ohe['Casado(a)'] = dados['Casado(a)'].copy()
df_ohe['Dependentes'] = dados['Dependentes'].copy()
df_ohe['ServicoTelefonico'] = dados['ServicoTelefonico'].copy()
df_ohe['MultiplasLinhas'] = dados['MultiplasLinhas'].copy()
df_ohe['SegurancaOnline'] = dados['SegurancaOnline'].copy()
df_ohe['BackupOnline'] = dados['BackupOnline'].copy()
df_ohe['ProtecaodeDispositivo'] = dados['ProtecaodeDispositivo'].copy()
df_ohe['SuporteTecnico'] = dados['SuporteTecnico'].copy()
df_ohe['StreamingTV'] = dados['StreamingTV'].copy()
df_ohe['StreamingFilmes'] = dados['StreamingFilmes'].copy()
df_ohe['CobrancasemPapel'] = dados['CobrancasemPapel'].copy()

In [75]:
# Analisando coposição do nosso Data Frame
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ServicodeInternet_DSL                7043 non-null   uint8  
 1   ServicodeInternet_Fibra optica       7043 non-null   uint8  
 2   ServicodeInternet_Nao                7043 non-null   uint8  
 3   Contrato_Anual                       7043 non-null   uint8  
 4   Contrato_Dois anos                   7043 non-null   uint8  
 5   Contrato_Mensal                      7043 non-null   uint8  
 6   MetododePagamento_Cartao de credito  7043 non-null   uint8  
 7   MetododePagamento_Correspondencia    7043 non-null   uint8  
 8   MetododePagamento_Debito automatico  7043 non-null   uint8  
 9   MetododePagamento_Eletronico         7043 non-null   uint8  
 10  ID                                   7043 non-null   object 
 11  CobrancasMensais              

In [76]:
#Construção de dicionario para alterar características binárias
binario = {'Sim': 1, 'Nao':0, 'Homem': 0, 'Mulher': 1}

In [86]:
df_ohe = df_ohe.replace(binario)
df_ohe.head()

,ServicodeInternet_DSL,ServicodeInternet_Fibra optica,ServicodeInternet_Nao,Contrato_Anual,Contrato_Dois anos,Contrato_Mensal,MetododePagamento_Cartao de credito,MetododePagamento_Correspondencia,MetododePagamento_Debito automatico,MetododePagamento_Eletronico,...,Dependentes,ServicoTelefonico,MultiplasLinhas,SegurancaOnline,BackupOnline,ProtecaodeDispositivo,SuporteTecnico,StreamingTV,StreamingFilmes,CobrancasemPapel
0,1,0,0,1,0,0,0,1,0,0,...,1,1,0,0,1,0,1,1,0,1
1,1,0,0,0,0,1,0,1,0,0,...,0,1,1,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
3,0,1,0,0,0,1,0,0,0,1,...,0,1,0,0,1,1,0,1,1,1
4,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,1,1,0,1
